In [1]:
import os   #cette ligneet celle du dessous sonts sensées trouver la pathing du fichier csv qu'on veut ouvrir avec PANDAS 
os.getcwd()
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from enum import Enum
import collections, numpy # count the occurrence of certain item in an ndarray in Python
sns.set()

pd.options.display.max_rows = 10000

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Lecture de notre data pricipal :

data = pd.read_csv('data/EdStatsData.csv')


In [3]:
# Selection des data pour l'année 2015:

data = data.drop(['Country Name','Unnamed: 69'], axis=1)
data = data.drop(data.loc[:,'1970':'2014'], axis=1)
data = data.drop(data.loc[:,'2016':'2100'], axis=1)
data = pd.melt(data,id_vars = ['Indicator Code','Indicator Name','Country Code',],var_name='Year')

# on drop seulement un seul indicateur d'un pays et pas tout les indics du pays 
data = data.dropna()

data.sample(5)



,Indicator Code,Indicator Name,Country Code,Year,value
741522,SE.SEC.ENRL.FE,"Enrolment in secondary education, female (number)",SSD,2015,56612.000000
318862,SE.PRM.TENR.MA,"Adjusted net enrolment rate, primary, male (%)",EST,2015,94.204498
260136,PRJ.POP.1519.S1.MA,Wittgenstein Projection: Population age 15-19 ...,COG,2015,59.150000
358950,PRJ.ATT.2039.2.MA,Wittgenstein Projection: Percentage of the pop...,DEU,2015,0.130000
817209,PRJ.ATT.ALL.1.FE,Wittgenstein Projection: Percentage of the tot...,TUN,2015,0.150000


In [12]:
# visualisation des indicateurs 

data['Indicator Name'].value_counts().sort_values(ascending=False)

Population growth (annual %)                                                                                                              232
Population, total                                                                                                                         232
Internet users (per 100 people)                                                                                                           223
Population, ages 15-64 (% of total)                                                                                                       218
GDP at market prices (current US$)                                                                                                        218
GDP per capita (current US$)                                                                                                              218
GDP at market prices (constant 2005 US$)                                                                                                  215
GDP pe

In [5]:
# L'idée ici c'est de prendre les indicateurs qui se sont repeter au moins 70% du nombres de pays disponible ( dans notre cas 242)

data=data.groupby('Indicator Code').filter(lambda x: x['Indicator Code'].size>130)

# Je selectionne les indicateurs interessants et qui ont la fréquence d'apparition la plus elevée(voir en haut)

test = data.loc[data['Indicator Code'].isin(['SP.POP.TOTL', 'IT.NET.USER.P2',
                                         'NY.GDP.PCAP.CD','SP.SEC.TOTL.IN',
                                         'SE.SEC.ENRL.GC','SP.POP.1524.TO.UN','PRJ.POP.ALL.3.MF','PRJ.POP.ALL.4.MF'])]

# j'ai visualiser avec un value counts les pays et j'enleve ceux qui se sont repetés moins6 fois car jai 8 indics pricipaux


test = test.groupby(['Country Code']).filter(lambda x : x['Country Code'].size>6)

test.sort_values('Country Code').sample(5)


,Indicator Code,Indicator Name,Country Code,Year,value
166122,SE.SEC.ENRL.GC,"Enrolment in secondary general, both sexes (nu...",BEN,2015,9.416890e+05
336048,SP.POP.TOTL,"Population, total",FIN,2015,5.479531e+06
239470,NY.GDP.PCAP.CD,GDP per capita (current US$),CHL,2015,1.365323e+04
588933,SP.POP.TOTL,"Population, total",MMR,2015,5.240367e+07
392146,PRJ.POP.ALL.3.MF,Wittgenstein Projection: Population in thousan...,GNB,2015,1.029600e+02


In [16]:
# j'ai mis tout les 8 indicateurs comme colonnes pour mieux visualiser chaque pays dans UNE SEULE ligne, donc on a toute les infos a chaque ligne

test2 = test.pivot('Country Code',columns='Indicator Name', values='value')
test2 = test2.reset_index()

# Vu qu'il ya trés peu de NaN a ce point j'ai decider de fill tout les NaN car ils ne vont pas impacter mes conclusions 

# test2.apply(lambda x: x.fillna(method = 'ffill',inplace=True))

test2['Enrolment in secondary general, both sexes (number)'].isnull().value_counts()




test2.info()

False    113
True      48
Name: Enrolment in secondary general, both sexes (number), dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 9 columns):
Country Code                                                                                                           161 non-null object
Enrolment in secondary general, both sexes (number)                                                                    113 non-null float64
GDP per capita (current US$)                                                                                           160 non-null float64
Internet users (per 100 people)                                                                                        161 non-null float64
Population of the official age for secondary education, both sexes (number)                                            159 non-null float64
Population, ages 15-24, total                                                                                          158 non-null float64
Population, total                                        

In [10]:
# je veux voir a partir de quel GDP on considére un pays comme potentiel 

test2.groupby('Country Code')['GDP per capita (current US$)'].mean().sort_values().describe()


count       160.000000
mean      13461.565920
std       18769.277390
min         300.676557
25%        1610.608034
50%        5105.641132
75%       15676.846927
max      101446.786303
Name: GDP per capita (current US$), dtype: float64

In [11]:
# sns.barplot(x="Country Code",y="Population of the official age for secondary education, both sexes (number)",data=test2)

# sns.distplot(test2['GDP per capita (current US$)'],kde=False)


In [ ]:
# sns.stripplot(x = 'GDP per capita (current US$)',y = 'Internet users (per 100 people)',data = test2)



In [ ]:
# sns.stripplot(y = 'GDP per capita (current US$)',x = 'Enrolment in secondary general, both sexes (number)',data = test2)